In [6]:
import json
import os
import requests
import zipfile

from datetime import datetime
from tqdm import tqdm

In [22]:
os.getcwd()

'd:\\tmp\\_reveng\\audioeffect-qm\\resae'

In [23]:
os.chdir('..')

In [21]:
src = 'resae/ugc.json'
d = json.load(open(src))

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for ae in tqdm(d['ugcAudioEffectItems']):

    ae_id = ae['effectId']
    ae_name = ae['effectName']
    ae_url = ae['effectDownloadLink']
    ae_ts = datetime.strptime(ae['createTime'], '%Y-%m-%d %H:%M:%S').timestamp()
    target = f'{ae_id:04d}-{ae_name}.zip'

    response = requests.get(ae_url)
    with open(target, 'wb') as f:
        f.write(response.content)
    os.utime(target, (ae_ts, ae_ts))

    zip_file = zipfile.ZipFile(target, 'r')
    zip_file.extractall(dst)
    zip_file.close()

    try:
        if len(os.listdir('AudioEffectPackage')) == 2:
            target_new = f'{ae_id:04d}-{ae_name}.qmaep'
            os.rename('AudioEffectPackage/AudioEffect.qmaep', target_new)
            os.utime(target_new, (ae_ts, ae_ts))
            os.remove(target)
    except:
        pass
    for f in os.listdir('AudioEffectPackage'):
        os.remove(os.path.join('AudioEffectPackage', f))

os.chdir('..')

100%|██████████| 11/11 [00:18<00:00,  1.64s/it]
